In [1]:
# import sys for python exception
import sys
# import connector library for postgresql
import psycopg2
# import library for data transform
import pandas as pd
# import the error handling for psycopg2
from psycopg2 import OperationalError
from psycopg2.extras import execute_batch

In [2]:
# Custom error handling

def db_error_tracing(err):
  # get detail exception
  err_type, traceback = sys.exc_info()
  # get line error
  err_line = traceback.tb_lineno
  # print connection error
  print("\npsycopg2 ERROR:", err, "on line number:", err_line)
  print("psycopg2 traceback:", traceback, "--- type", err_type)

In [3]:
# Melakukan percobaan koneksi

def db_connection():
  conn = None
  try:
    print("Connecting to the PostgreSQL!")
    # Set the connection parameters
    conn = psycopg2.connect (
      database = "KALBE",
      host = "localhost",
      port = "5432",
      user = "postgres",
      password  = "k/V3}>fQ[kyazX"
    )
    print("Connecting successful!")
    
  except OperationalError as err:
    # call function for error
    db_error_tracing(err)
    # Rollback database if connection was fail
    conn.rollback()

  return conn


In [4]:
# Melakukan pembuatan table baru di dalam postgreSQL

def create_postgres_tables():
  # Connect to the database
  conn = db_connection()
  conn.autocommit = True
  # Membuat object cursor koneksi
  cursor = conn.cursor()
  
  try:        
    # Dropping table iris if exists
    cursor.execute("DROP TABLE IF EXISTS inventory")
    
    # Creating a table
    cursor.execute("""CREATE TABLE inventory (item_code INTEGER PRIMARY KEY, item_name VARCHAR(25) NOT NULL, item_price INTEGER NOT NULL, item_total INTEGER NOT NULL, Row_id INTEGER NOT NULL) """)    
    print("inventory table is created successfully!")
    
    conn.commit()
    # Closing the cursor & connection
    cursor.close()
    conn.close()
  
  except OperationalError as err:
    # pass exception to function
    db_error_tracing(err)
    # Rollback database if connection was fail
    conn.rollback()
    # Set the close to cursor
    cursor.close()

In [16]:
# Melakukan pembuatan table baru di dalam postgreSQL

def create_postgres_tables_2():
  # Connect to the database
  conn = db_connection()
  conn.autocommit = True
  # Membuat object cursor koneksi
  cursor = conn.cursor()
  
  try:        
    # Dropping table iris if exists
    cursor.execute("DROP TABLE IF EXISTS customer_transaction")
    
    # Creating a table
    cursor.execute("""CREATE TABLE customer_transaction (order_no INTEGER NOT NULL, purchase_amount INTEGER NOT NULL, order_date DATE NOT NULL DEFAULT CURRENT_DATE, customer_id INTEGER NOT NULL, salesman_id INTEGER NOT NULL, Row_id INTEGER NOT NULL)""")    
    print("customer_transaction table is created successfully!")
    
    conn.commit()
    # Closing the cursor & connection
    cursor.close()
    conn.close()
  
  except OperationalError as err:
    # pass exception to function
    db_error_tracing(err)
    # Rollback database if connection was fail
    conn.rollback()
    # Set the close to cursor
    cursor.close()

In [5]:
df = pd.read_csv('Kalbe-Dataset-1.csv')
df['Row_id'] = range(1, 1+len(df))
df.head()

,Item_code,Item_name,Item_price,Item_total,Row_id
0,2341,Promag Tablet,3000,100,1
1,2342,Hydro Coco 250ML,7000,20,2
2,2343,Nutrive Benecol 100ML,20000,30,3
3,2344,Blackmores Vit C 500Mg,95000,45,4
4,2345,Entrasol Gold 370G,90000,120,5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Item_code   5 non-null      int64 
 1   Item_name   5 non-null      object
 2   Item_price  5 non-null      int64 
 3   Item_total  5 non-null      int64 
 4   Row_id      5 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 328.0+ bytes


In [7]:
df_v1 = pd.read_csv('Kalbe-Dataset-2.csv')
df_v1['Row_id'] = range(1, 1+len(df_v1))
df_v1.head()

,order_no,purchase_amount,order_date,customer_id,salesman_id,Row_id
0,10001,150,2022-10-05,2005,3002,1
1,10009,279,2022-09-10,2001,3005,2
2,10002,65,2022-10-05,2002,3001,3
3,10004,110,2022-08-17,2009,3003,4
4,10007,948,2022-09-10,2005,3002,5


In [8]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   order_no         6 non-null      int64 
 1   purchase_amount  6 non-null      int64 
 2   order_date       6 non-null      object
 3   customer_id      6 non-null      int64 
 4   salesman_id      6 non-null      int64 
 5   Row_id           6 non-null      int64 
dtypes: int64(5), object(1)
memory usage: 416.0+ bytes


In [9]:
# Melakukan penyimpanan data dari .csv ke postgreSQL

def write_to_postgres():
    # Connect to the database
    conn = db_connection()
    conn.autocommit = True
    cursor = conn.cursor()

    try:        
      inserted_row_count = 0
      
      for _, row in df.iterrows():
          count_query = f"""SELECT COUNT(*) FROM inventory WHERE Row_id = {row['Row_id']}"""
          cursor.execute(count_query)
          result = cursor.fetchone()
          
          if result[0] == 0:
            inserted_row_count += 1
            cursor.execute("""INSERT INTO inventory (item_code, item_name, item_price, item_total, Row_id
                           ) VALUES (%s, %s, %s, %s, %s)""", 
                           (int(row[0]), str(row[1]), str(row[2]), int(row[3]), int(row[4])))
      
      print("Data convert to dataFrame successful!")
      conn.commit()
      # Closing the cursor & connection
      cursor.close()
      conn.close()
    
    except OperationalError as err:
        # call function for error
        db_error_tracing(err)
        # Rollback database if connection was fail
        conn.rollback()
        # Set the close to cursor
        cursor.close()

In [19]:
# Melakukan penyimpanan data dari .csv ke postgreSQL

def write_to_postgres_2():
    # Connect to the database
    conn = db_connection()
    conn.autocommit = True
    cursor = conn.cursor()

    try:        
      inserted_row_count = 0
      
      for _, row in df_v1.iterrows():
          count_query = f"""SELECT COUNT(*) FROM customer_transaction WHERE Row_id = {row['Row_id']}"""
          cursor.execute(count_query)
          result = cursor.fetchone()
          
          if result[0] == 0:
            inserted_row_count += 1
            cursor.execute("""INSERT INTO customer_transaction (order_no, purchase_amount, order_date, customer_id, salesman_id, Row_id
                           ) VALUES (%s, %s, %s, %s, %s, %s)""", 
                           (int(row[0]), str(row[1]), str(row[2]), int(row[3]), int(row[4]), int(row[5])))
      
      print("Data convert to dataFrame successful!")
      conn.commit()
      # Closing the cursor & connection
      cursor.close()
      conn.close()
    
    except OperationalError as err:
        # call function for error
        db_error_tracing(err)
        # Rollback database if connection was fail
        conn.rollback()
        # Set the close to cursor
        cursor.close()

In [ ]:
# Menghubah spesific data dari spesific table di dalam database

In [ ]:
# Menghapus table dari spesific database

In [10]:
#--- Deklarasi SQL query untuk menampilkan data

# Connect to the database
conn = db_connection()
conn.autocommit = True

# Execute method
create_postgres_tables()
write_to_postgres()

# Declare cursor for connection
cursor = conn.cursor()

# Execute query
query = "SELECT * FROM Inventory"
cursor.execute(query)

# Fetch all the records
tuples = cursor.fetchall()

# Prepare list of columns for dataFrame
cols = list(df.columns)

pharma_df = pd.DataFrame(tuples, columns=cols)
display(pharma_df.head())

# Close cursor
cursor.close()

# Close the connection
conn.close()

Connecting to the PostgreSQL!
Connecting successful!
Connecting to the PostgreSQL!
Connecting successful!
inventory table is created successfully!
Connecting to the PostgreSQL!
Connecting successful!
Data convert to dataFrame successful!


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\27345185.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (int(row[0]), str(row[1]), str(row[2]), int(row[3]), int(row[4])))


,Item_code,Item_name,Item_price,Item_total,Row_id
0,2341,Promag Tablet,3000,100,1
1,2342,Hydro Coco 250ML,7000,20,2
2,2343,Nutrive Benecol 100ML,20000,30,3
3,2344,Blackmores Vit C 500Mg,95000,45,4
4,2345,Entrasol Gold 370G,90000,120,5


In [20]:
#--- Deklarasi SQL query untuk menampilkan data

# Connect to the database
conn = db_connection()
conn.autocommit = True

# Execute method
create_postgres_tables_2()
write_to_postgres_2()

# Declare cursor for connection
cursor = conn.cursor()

# Execute query
query = "SELECT * FROM customer_transaction"
cursor.execute(query)

# Fetch all the records
tuples = cursor.fetchall()

# Prepare list of columns for dataFrame
cols = list(df_v1.columns)

pharma_df = pd.DataFrame(tuples, columns=cols)
display(pharma_df.head())

# Close cursor
cursor.close()

# Close the connection
conn.close()

Connecting to the PostgreSQL!
Connecting successful!
Connecting to the PostgreSQL!
Connecting successful!
customer_transaction table is created successfully!
Connecting to the PostgreSQL!
Connecting successful!
Data convert to dataFrame successful!


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\394683846.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (int(row[0]), str(row[1]), str(row[2]), int(row[3]), int(row[4]), int(row[5])))


,order_no,purchase_amount,order_date,customer_id,salesman_id,Row_id
0,10001,150,2022-10-05,2005,3002,1
1,10009,279,2022-09-10,2001,3005,2
2,10002,65,2022-10-05,2002,3001,3
3,10004,110,2022-08-17,2009,3003,4
4,10007,948,2022-09-10,2005,3002,5


In [11]:
# Connect to the database
conn = db_connection()
conn.autocommit = True

# Declare cursor for connection
cursor = conn.cursor()

# Execute query
query = "SELECT item_name FROM Inventory WHERE item_total = (SELECT MAX(item_total) FROM Inventory LIMIT 1)"
show_highest_item = pd.read_sql_query(query, conn)
print(show_highest_item)

# Close cursor
cursor.close()

# Close the connection
conn.close()


Connecting to the PostgreSQL!
Connecting successful!
            item_name
0  Entrasol Gold 370G


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\420375519.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  show_highest_item = pd.read_sql_query(query, conn)


In [12]:
# Connect to the database
conn = db_connection()
conn.autocommit = True

# Declare cursor for connection
cursor = conn.cursor()

# Execute query #1- update item_price value
update_item_price = 125000
query = f"UPDATE Inventory SET item_price= {update_item_price} WHERE item_total = (SELECT MAX(item_total) FROM Inventory LIMIT 1)"
cursor.execute(query)
print("Update item_price value successful!")

# Execute query #2- check data in the table
query = "SELECT * FROM Inventory"
show_update_item = pd.read_sql_query(query, conn)
print(show_update_item)

# Close cursor
cursor.close()

# Close the connection
conn.close()

Connecting to the PostgreSQL!
Connecting successful!
Update item_price value successful!
   item_code               item_name  item_price  item_total  row_id
0       2341           Promag Tablet        3000         100       1
1       2342        Hydro Coco 250ML        7000          20       2
2       2343   Nutrive Benecol 100ML       20000          30       3
3       2344  Blackmores Vit C 500Mg       95000          45       4
4       2345      Entrasol Gold 370G      125000         120       5


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\1275475988.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  show_update_item = pd.read_sql_query(query, conn)


In [13]:
#--- Menginput data baru ke dalam table

# Connect to the database
conn = db_connection()
conn.autocommit = True

# Declare cursor for connection
cursor = conn.cursor()

# Execute query #1- insert new data into table row
# Input data as list of dicts and using named parameters to avoid duplicating data.
input_new_data = [{'item_code': 2343, 'item_name': 'Vicks F44', 'item_price': 25000, 'item_total': 25, 'row_id': 6}]
# Using execute_batch to inserts using a multi-line statement
execute_batch(cursor, 'INSERT INTO Inventory values(%(item_code)s, %(item_name)s, %(item_price)s, %(item_total)s, %(row_id)s)', input_new_data)

# Execute query #2- check data in the table
query = "SELECT * FROM Inventory"
show_update_item = pd.read_sql_query(query, conn)
print(show_update_item)

# Close cursor
cursor.close()

# Close the connection
conn.close()

Connecting to the PostgreSQL!
Connecting successful!


UniqueViolation: duplicate key value violates unique constraint "inventory_pkey"
DETAIL:  Key (item_code)=(2343) already exists.


In [14]:
#--- Menghapus data baru ke dalam table

# Connect to the database
conn = db_connection()
conn.autocommit = True

# Declare cursor for connection
cursor = conn.cursor()

# Execute query #1- delete a spesific data (row)
query = "DELETE FROM Inventory WHERE item_total = (SELECT MIN(item_total) FROM Inventory LIMIT 1)"
cursor.execute(query)
print("Delete item_name value successful!")

# Execute query #2- check data in the table
query = "SELECT * FROM Inventory"
show_update_item = pd.read_sql_query(query, conn)
print(show_update_item)

# Close cursor
cursor.close()

# Close the connection
conn.close()

Connecting to the PostgreSQL!
Connecting successful!
Delete item_name value successful!
   item_code               item_name  item_price  item_total  row_id
0       2341           Promag Tablet        3000         100       1
1       2343   Nutrive Benecol 100ML       20000          30       3
2       2344  Blackmores Vit C 500Mg       95000          45       4
3       2345      Entrasol Gold 370G      125000         120       5


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\1865614990.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  show_update_item = pd.read_sql_query(query, conn)


In [21]:
#--- Menampilkan semua daftar customer yang melakukan pembelian barang kurang dari
#--- 100 item atau customer yang melakukan pemesanan lewat dari tanggal 25 Agust 2022
#--- dan customer_id lebih besar dari 2001

# Connect to the database
conn = db_connection()
conn.autocommit = True

# Declare cursor for connection
cursor = conn.cursor()

# Execute query
query = "SELECT * FROM customer_transaction WHERE (purchase_amount < 100) OR (order_date > '2022-08-25' AND customer_id > 2001)"
show_customer_data = pd.read_sql_query(query, conn)
print(show_customer_data)

# Close cursor
cursor.close()

# Close the connection
conn.close()


Connecting to the PostgreSQL!
Connecting successful!
   order_no  purchase_amount  order_date  customer_id  salesman_id  row_id
0     10001              150  2022-10-05         2005         3002       1
1     10002               65  2022-10-05         2002         3001       3
2     10007              948  2022-09-10         2005         3002       5


C:\Users\RickyS-PC\AppData\Local\Temp\ipykernel_15332\501005571.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  show_customer_data = pd.read_sql_query(query, conn)
